In [16]:
from __future__ import print_function
from unicodecsv import DictReader
import pandas as pd
from paths import DATASETS_DIR, DUMPS_DIR
from pprint import pprint
from data import generate_dump
import os
from collections import OrderedDict
# import mysql.connector
from sqlalchemy import create_engine
# import pyodbc
from helpers import freq_iso_to_pandas
from pydatajson.helpers import parse_repeating_time_interval_to_days
from generate_dumps import save_to_dta
import numpy as np

In [5]:
dump_path = os.path.join(DUMPS_DIR, "series-tiempo.{}")

## Genero dataframe con el dump

In [6]:
%time df = generate_dump()

CPU times: user 31.6 s, sys: 1.86 s, total: 33.5 s
Wall time: 33.8 s


In [7]:
drop_cols = ["distribucion_indice_tiempo", "valor"]
index_cols = ["dataset_id", "distribucion_id", "serie_id"]
series_cols = df.columns.drop(drop_cols)
series_group = df.groupby(index_cols)

In [8]:
df_series = df.drop(drop_cols, axis=1).drop_duplicates().set_index(index_cols)

In [9]:
def _get_valor_anterior_anio(series_dataframe):
    series = pd.Series(list(series_dataframe.valor), list(series_dataframe.distribucion_indice_tiempo)).sort_index()
    return series.get(series.index[-1] - pd.DateOffset(years=1))

In [10]:
# rango temporal de la serie
df_series["serie_indice_inicio"] = series_group["distribucion_indice_tiempo"].min()
df_series["serie_indice_final"] = series_group["distribucion_indice_tiempo"].max()
df_series["serie_valores_cant"] = series_group["distribucion_indice_tiempo"].count()

# estado de actualización de los datos
df_series["serie_dias_no_cubiertos"] = df_series.apply(
    lambda x: (pd.datetime.now() - pd.to_datetime(x["serie_indice_final"]).to_period(
        freq_iso_to_pandas("R/P3M", how="end")).to_timestamp(how="end")).days,
    axis=1)
df_series["serie_actualizada"] = df_series.apply(
    lambda x: x["serie_dias_no_cubiertos"] < 2 * parse_repeating_time_interval_to_days(x["distribucion_indice_frecuencia"]),
    axis=1)

# valores representativos nominales
df_series["valor_ultimo"] = series_group.apply(
    lambda x: x.loc[x.distribucion_indice_tiempo.argmax(), "valor"])
df_series["valor_anterior"] = series_group.apply(
    lambda x: pd.Series(list(x.valor), list(x.distribucion_indice_tiempo)).sort_index()[-2]
)
df_series["valor_anterior_anio"] = series_group.apply(_get_valor_anterior_anio)

# valores representativos en variación porcentual
df_series["var_pct_anterior"] = df_series["valor_ultimo"] / df_series["valor_anterior"] - 1
df_series["var_pct_anterior_anio"] = df_series["valor_ultimo"] / df_series["valor_anterior_anio"] - 1

In [12]:
df_series.to_excel("series-tiempo-resumen.xlsx", "resumen", index=True, index_label=True, merge_cells=False)

In [28]:
df_stata = df_series

In [33]:
for col in df_series.columns:
    
    # limita el largo de los campos de texto
    if df_stata[col].dtype.name == "object":
        df_stata[col] = df_stata[col].str[:244]
    
    # limita la precisión de los números decimales
    elif "float" in df_stata[col].dtype.name:
        df_stata[col] = df_stata[col].astype(np.float32)

In [34]:
df_series.to_stata("series-tiempo-resumen.dta", write_index=False)

ValueError: Column valor_ultimo has a maximum value of infinity which is outside the range supported by Stata.

In [35]:
np.isinf(2.010)

False

## Genero distribuciones de distintos formatos

In [ ]:
%%time
# CSV
path = dump_path.format("csv")
df.to_csv(path, encoding="utf-8", sep=",", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# EXCEL
path = dump_path.format("xlsx")
writer = pd.ExcelWriter(path, engine='xlsxwriter')
df.to_excel(writer, "data", merge_cells=False, encoding="utf-8", index=False)
writer.save()
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQLITE
path = dump_path.format("db")
engine = create_engine('sqlite:///{}'.format(path), echo=True)
df.to_sql("series_tiempo", engine, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# ACCESS
# set up some constants
DRV = '{Microsoft Access Driver (*.mdb)}'; PWD = 'pw'

# connect to db
path = dump_path.format("mdb")
con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV, path, PWD))
cur = con.cursor()

df.to_sql("series_tiempo", con, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQL
# la gracia acá es hacer bien las tablas con sus índices y pensar si MySQL, Postgresql o SQLite
path = dump_path.format("sql")
df.to_sql(path, "data", merge_cells=False, encoding="utf-8", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [22]:
%%time 
# DTA
# hay que ver cómo manejar el límite de caracteres
path = dump_path.format("dta")
df_stata = df.copy()
for col in df_stata.columns:
    if df_stata[col].dtype.name == "object":
        # trunca campos de texto a 244 caracteres aceptados por STATA
        df_stata[col] = df_stata[col].str[:244]
df_stata.to_stata(path, write_index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

1097MB
CPU times: user 20.1 s, sys: 5.21 s, total: 25.3 s
Wall time: 25.6 s


In [ ]:
df.head()